<div class='alert alert-block' style='background-color: #ffffff; border: 1px solid #333333; border-radius: 10px; padding: 20px; text-align: center;'> <h1 style='font-family: Arial, sans-serif; color: #000000; font-size: 36px;'>MACHINE LEARNING</h1> <h2 style='font-family: Arial, sans-serif; color: #333333; font-size: 28px;'>MDSAA-BA Program</h2> <hr style='border: 1px solid #555555;'> <h3 style='font-family: 'Open Sans', sans-serif; color: #666666; font-size: 24px;'>To Grant Or Not To Grant Project 2024/2025</h3> </div>

## Table of contents

1. [Importing Libraries](#1.-Importing-Libraries)
2. [Importing the data](#2.-Importing-the-data)
3. [Exploring the data](#3.-Exploring-the-data)

## 1. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings   
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

warnings.filterwarnings('ignore')

<br>

## 2. Importing the data

The `display.max_columns` option controls the number of columns to be printed. By setting the `display.max_columns` option to `None`, Pandas will display all columns in the DataFrame.

In [2]:
# Displays all columns of the dataset
pd.set_option('display.max_columns', None)

In [3]:
train = pd.read_csv('../Data/train_explored.csv', index_col=0)
test = pd.read_csv('../Data/test_explored.csv', index_col=0)

<br>

Ensuring that the date columns are in datetime again.

In [4]:
train['Accident Date'] = pd.to_datetime(train['Accident Date'])
test['Accident Date'] = pd.to_datetime(test['Accident Date'])

In [5]:
train['Assembly Date'] = pd.to_datetime(train['Assembly Date'])
test['Assembly Date'] = pd.to_datetime(test['Assembly Date'])

In [6]:
train['C-2 Date'] = pd.to_datetime(train['C-2 Date'])
test['C-2 Date'] = pd.to_datetime(test['C-2 Date'])

In [7]:
train['C-3 Date'] = pd.to_datetime(train['C-3 Date'])
test['C-3 Date'] = pd.to_datetime(test['C-3 Date'])

<br>

## 3. Model Selection

Hold out method

In this approach we randomly split the complete data into training and test sets. Then we perform the model training on the training set and use the validation set for validation purpose. 80% of the data will be used for training purposes, while 20% for validation.

We first split between independent and dependent variables.

In [8]:
x = train.drop(columns= 'Claim Injury Type')
y = train['Claim Injury Type']

In [9]:
train_set, val_set, y_train, y_val = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    shuffle=True, 
                                                    stratify=y
                                                   )

<br>

##### 3.1 Addressing the inconsistencies found

<br>

##### 3.1.1 `Assembly Date`

First, its necessary to understand if there is any misalignment between these features:
- `Assembly Date` and `Accident Date`: `Assembly Date` should be after `Accident Date`, since it would not make sense for the assembly process to occur before the accident;
- `C-2 Date` and `Accident Date`: `C-2 Date` should also be after `Accident Date`, since its receipt of the Employer's Report of Work-Related Injury

In [10]:
invalid_assembly_dates_train = train_set[(train_set['Assembly Date'].notnull()) & 
                           (train_set['Accident Date'].notnull()) & 
                           (train_set['Assembly Date'] < train_set['Accident Date'])]
print(f"Invalid Assembly Dates: {invalid_assembly_dates_train.shape[0]}")

Invalid Assembly Dates: 1153


In [11]:
invalid_assembly_dates_val = val_set[(val_set['Assembly Date'].notnull()) & 
                                      (val_set['Accident Date'].notnull()) & 
                                      (val_set['Assembly Date'] < val_set['Accident Date'])]
print(f"Invalid Assembly Dates in val_set: {invalid_assembly_dates_val.shape[0]}")

Invalid Assembly Dates in val_set: 254


In [12]:
invalid_assembly_dates_test = test[(test['Assembly Date'].notnull()) & 
                                    (test['Accident Date'].notnull()) & 
                                    (test['Assembly Date'] < test['Accident Date'])]
print(f"Invalid Assembly Dates in test: {invalid_assembly_dates_test.shape[0]}")

Invalid Assembly Dates in test: 222


In [13]:
train_set[train_set['Assembly Date'] < train_set['Accident Date']].head(5)

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Agreement Reached,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,
5615301,2021-02-26,45.0,N,2020-12-10,Y,0.000000,1976.0,2021-04-14,2020-12-04,AMERICAN ZURICH INSURANCE CO,1A. PRIVATE,KINGS,N,NYC,2021-06-11,M,1.0,42.0,IV,97.0,52.0,35.0,NaN,0.0,3.0
5677270,2021-04-21,64.0,N,2021-03-11,Y,0.000000,1957.0,2021-09-28,2021-03-05,AMERICAN ZURICH INSURANCE CO,1A. PRIVATE,NASSAU,N,NYC,2022-07-22,M,0.0,23.0,IV,56.0,59.0,-9.0,11561,0.0,2.0
5526108,2020-07-29,50.0,N,2020-07-28,Y,3.342103,1970.0,2020-07-27,2020-08-06,A I U INSURANCE COMPANY,1A. PRIVATE,NASSAU,N,NYC,2020-10-27,M,1.0,48.0,IV,16.0,78.0,35.0,11801,0.0,4.0
5439827,2020-03-21,63.0,N,2020-02-28,Y,3.430420,1956.0,2020-03-13,2020-02-25,NYC TRANSIT AUTHORITY,3A. SELF PUBLIC,KINGS,N,NYC,2021-08-16,M,2.0,48.0,UK,99.0,31.0,13.0,07731,0.0,0.0
5548633,2020-09-23,33.0,N,2020-08-31,Y,3.388027,1987.0,2021-12-08,2020-08-26,"CNY OTHER THAN ED, HED WATER",3A. SELF PUBLIC,RICHMOND,N,NYC,2022-03-25,M,2.0,92.0,IV,97.0,71.0,51.0,11209,0.0,1.0


Given that there are 1153 rows with this invalid difference, we considered swap the values of the `Assembly Date` and `Accident Date` columns for the rows where the invalid condition is met.

In [14]:
train_set.loc[invalid_assembly_dates_train.index, ['Assembly Date', 'Accident Date']] = \
    train_set.loc[invalid_assembly_dates_train.index, ['Accident Date', 'Assembly Date']].values

In [15]:
print(f"Invalid Assembly Dates after correction: {train_set[(train_set['Assembly Date'] < train_set['Accident Date'])].shape[0]}")

Invalid Assembly Dates after correction: 0


In [16]:
val_set.loc[invalid_assembly_dates_val.index, ['Assembly Date', 'Accident Date']] = \
    val_set.loc[invalid_assembly_dates_val.index, ['Accident Date', 'Assembly Date']].values

In [17]:
test.loc[invalid_assembly_dates_test.index, ['Assembly Date', 'Accident Date']] = \
    test.loc[invalid_assembly_dates_test.index, ['Accident Date', 'Assembly Date']].values

For now, are are going to create a feature that will have the difference in days of the `Assembly Date` and `Accident Date`.

In [18]:
train_set['Days_to_Assembly'] = (train_set['Assembly Date'] - train_set['Accident Date']).dt.days
val_set['Days_to_Assembly'] = (val_set['Assembly Date'] - val_set['Accident Date']).dt.days
test['Days_to_Assembly'] = (test['Assembly Date'] - test['Accident Date']).dt.days

<br>

##### 3.1.2 `C-2 Date` 

In [19]:
invalid_c2_dates_train = train_set[(train_set['C-2 Date'].notnull()) & 
                           (train_set['Accident Date'].notnull()) & 
                           (train_set['C-2 Date'] < train_set['Accident Date'])]
print(f"Invalid C2 Dates: {invalid_c2_dates_train.shape[0]}")

Invalid C2 Dates: 137


In [20]:
train_set[train_set['C-2 Date'] < train_set['Accident Date']].head(5)

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Agreement Reached,Number of Dependents,Days_to_Assembly
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,
5526108,2020-07-28,50.0,N,2020-07-29,Y,3.342103,1970.0,2020-07-27,2020-08-06,A I U INSURANCE COMPANY,1A. PRIVATE,NASSAU,N,NYC,2020-10-27,M,1.0,48.0,IV,16.0,78.0,35.0,11801,0.0,4.0,1.0
5407588,2019-11-07,35.0,N,2020-01-21,Y,3.364759,1984.0,2019-10-01,2020-01-13,"CNY OTHER THAN ED, HED WATER",3A. SELF PUBLIC,NEW YORK,N,NYC,2020-10-26,M,1.0,92.0,IV,97.0,52.0,35.0,10027,0.0,1.0,75.0
5559810,2020-09-17,56.0,N,2020-10-06,Y,0.000000,1964.0,2010-03-11,2020-09-15,STATE INSURANCE FUND,2A. SIF,ORANGE,N,ALBANY,2020-10-22,M,1.0,23.0,III,6.0,60.0,60.0,10930,0.0,1.0,19.0
5418954,2019-07-15,60.0,N,2020-02-03,Y,0.000000,NaN,2019-05-17,2020-01-29,"CNY OTHER THAN ED, HED WATER",3A. SELF PUBLIC,RICHMOND,N,NYC,2020-03-30,F,0.0,92.0,IV,97.0,78.0,35.0,10303,0.0,1.0,203.0
5430016,2020-01-06,55.0,N,2020-02-14,Y,0.000000,1964.0,2019-12-28,2020-03-18,AMERICAN CASUALTY CO,1A. PRIVATE,SUFFOLK,N,HAUPPAUGE,2020-03-26,M,0.0,23.0,IV,60.0,31.0,13.0,11704,1.0,2.0,39.0


In [21]:
invalid_c2_dates_val = val_set[(val_set['C-2 Date'].notnull()) & 
                                      (val_set['Accident Date'].notnull()) & 
                                      (val_set['C-2 Date'] < val_set['Accident Date'])]
print(f"Invalid C2 Dates in val_set: {invalid_c2_dates_val.shape[0]}")

Invalid C2 Dates in val_set: 28


In [22]:
invalid_c2_dates_test = test[(test['C-2 Date'].notnull()) & 
                                    (test['Accident Date'].notnull()) & 
                                    (test['C-2 Date'] < test['Accident Date'])]
print(f"Invalid C2 Dates in test: {invalid_c2_dates_test.shape[0]}")

Invalid C2 Dates in test: 29


Given that there are 137 rows with this invalid difference, we considered swap the values of the `C-2 Date` and `Accident Date` columns for the rows where the invalid condition is met.

In [23]:
train_set.loc[invalid_c2_dates_train.index, ['C-2 Date', 'Accident Date']] = \
    train_set.loc[invalid_c2_dates_train.index, ['Accident Date', 'C-2 Date']].values

In [24]:
print(f"Invalid C-2 Dates after correction: {train_set[(train_set['C-2 Date'] < train_set['Accident Date'])].shape[0]}")

Invalid C-2 Dates after correction: 0


In [25]:
val_set.loc[invalid_c2_dates_val.index, ['C-2 Date', 'Accident Date']] = \
    val_set.loc[invalid_c2_dates_val.index, ['Accident Date', 'C-2 Date']].values

In [26]:
test.loc[invalid_c2_dates_test.index, ['C-2 Date', 'Accident Date']] = \
    test.loc[invalid_c2_dates_test.index, ['Accident Date', 'C-2 Date']].values

<br>

#### 3.1.3 `C-3 Date`

In [27]:
invalid_c3_dates_train = train_set[(train_set['C-3 Date'].notnull()) & 
                           (train_set['Accident Date'].notnull()) & 
                           (train_set['C-3 Date'] < train_set['Accident Date'])]
print(f"Invalid C3 Dates: {invalid_c3_dates_train.shape[0]}")

Invalid C3 Dates: 983


In [28]:
train_set[train_set['C-3 Date'] < train_set['Accident Date']].head(5)

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Agreement Reached,Number of Dependents,Days_to_Assembly
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,
5615301,2020-12-10,45.0,N,2021-02-26,Y,0.000000,1976.0,2021-04-14,2020-12-04,AMERICAN ZURICH INSURANCE CO,1A. PRIVATE,KINGS,N,NYC,2021-06-11,M,1.0,42.0,IV,97.0,52.0,35.0,NaN,0.0,3.0,78.0
5677270,2021-03-11,64.0,N,2021-04-21,Y,0.000000,1957.0,2021-09-28,2021-03-05,AMERICAN ZURICH INSURANCE CO,1A. PRIVATE,NASSAU,N,NYC,2022-07-22,M,0.0,23.0,IV,56.0,59.0,-9.0,11561,0.0,2.0,41.0
5439827,2020-02-28,63.0,N,2020-03-21,Y,3.430420,1956.0,2020-03-13,2020-02-25,NYC TRANSIT AUTHORITY,3A. SELF PUBLIC,KINGS,N,NYC,2021-08-16,M,2.0,48.0,UK,99.0,31.0,13.0,07731,0.0,0.0,22.0
5548633,2020-08-31,33.0,N,2020-09-23,Y,3.388027,1987.0,2021-12-08,2020-08-26,"CNY OTHER THAN ED, HED WATER",3A. SELF PUBLIC,RICHMOND,N,NYC,2022-03-25,M,2.0,92.0,IV,97.0,71.0,51.0,11209,0.0,1.0,23.0
5885748,2021-12-27,51.0,N,2022-02-03,Y,3.428541,1970.0,2022-01-10,2021-12-13,A I U INSURANCE COMPANY,1A. PRIVATE,RICHMOND,N,NYC,2022-06-08,M,0.0,51.0,IV,97.0,78.0,35.0,10312,0.0,5.0,38.0


In [29]:
invalid_c3_dates_val = val_set[(val_set['C-3 Date'].notnull()) & 
                                      (val_set['Accident Date'].notnull()) & 
                                      (val_set['C-3 Date'] < val_set['Accident Date'])]
print(f"Invalid C3 Dates in val_set: {invalid_c3_dates_val.shape[0]}")

Invalid C3 Dates in val_set: 219


In [30]:
invalid_c3_dates_test = test[(test['C-3 Date'].notnull()) & 
                                    (test['Accident Date'].notnull()) & 
                                    (test['C-3 Date'] < test['Accident Date'])]
print(f"Invalid C3 Dates in test: {invalid_c3_dates_test.shape[0]}")

Invalid C3 Dates in test: 211


Given that there are 983 rows with this invalid difference, we considered swap the values of the `C-3 Date` and `Accident Date` columns for the rows where the invalid condition is met.

In [31]:
train_set.loc[invalid_c3_dates_train.index, ['C-3 Date', 'Accident Date']] = \
    train_set.loc[invalid_c3_dates_train.index, ['Accident Date', 'C-3 Date']].values

In [32]:
print(f"Invalid C-3 Dates after correction: {train_set[(train_set['C-3 Date'] < train_set['Accident Date'])].shape[0]}")

Invalid C-3 Dates after correction: 0


In [33]:
val_set.loc[invalid_c3_dates_val.index, ['C-3 Date', 'Accident Date']] = \
    val_set.loc[invalid_c3_dates_val.index, ['Accident Date', 'C-3 Date']].values

In [34]:
test.loc[invalid_c3_dates_test.index, ['C-3 Date', 'Accident Date']] = \
    test.loc[invalid_c3_dates_test.index, ['Accident Date', 'C-3 Date']].values

<br>

##### 3.1.4 `Accident Date`

In order to have the different components of the date (year, month and day), we are going to split the column `Accident Date` into three columns. We then proceed to remove the column from the datasets.

In [35]:
train_set['Accident Year'] = train_set['Accident Date'].apply(lambda x: x.year)
val_set['Accident Year'] = val_set['Accident Date'].apply(lambda x: x.year)
test['Accident Year'] = test['Accident Date'].apply(lambda x: x.year)

In [36]:
train_set['Accident Month'] = train_set['Accident Date'].apply(lambda x: x.month)
val_set['Accident Month'] = val_set['Accident Date'].apply(lambda x: x.month)
test['Accident Month'] = test['Accident Date'].apply(lambda x: x.month)

In [37]:
train_set['Accident Day'] = train_set['Accident Date'].apply(lambda x: x.day)
val_set['Accident Day'] = val_set['Accident Date'].apply(lambda x: x.day)
test['Accident Day'] = test['Accident Date'].apply(lambda x: x.day)

In [38]:
train_set = train_set.drop(columns = 'Accident Date')
val_set = val_set.drop(columns = 'Accident Date')
test = test.drop(columns = 'Accident Date')

<br>

1) Difference between `Accident Year` and `Age at Injury`

One aspect that is important to check is if the age at injury aligns with the difference between the accident date and the birth year. We had already seen that the feature `Age at Injury` has inconsistent values, therefore this mismatch might occur.

In [39]:
train_set.apply(lambda row: row['Accident Year'] - row['Birth Year'] == row['Age at Injury'], axis=1).value_counts()

False    240639
True     218581
Name: count, dtype: int64

From the output above, we can see that in 21 8517 claims, there is a misalignment between the calculated age and the recorded age at the time of injury.

In [40]:
(train_set['Accident Year'] - train_set['Birth Year'] - train_set['Age at Injury']).value_counts()

 0.0      218581
 1.0      194354
-1.0         301
 119.0        67
 118.0         6
-10.0          4
-2.0           4
-23.0          3
-3.0           3
-8.0           2
-7.0           2
-4.0           2
-13.0          2
 120.0         2
-18.0          2
-21.0          2
-27.0          1
-11.0          1
-6.0           1
-17.0          1
-15.0          1
-19.0          1
-14.0          1
Name: count, dtype: int64

The output above illustrates the differences in ages calculated from the `Accident Year`, `Birth Year`, and `Age at Injury`. Most differences are minor, with the majority falling within a 1-year difference. However, there are also significant anomalies where some records show a difference of more than 100 years. Also, there is some values that cannot be calculated due to missing values in some of these columns.

Given the inconsistencies and missing values in the date related data, these issues will be addressed. Since, some rows have mismatch between the calculated age and the recorded age at the time of injury we update the age based on the accident year and birth year

In [41]:
train_set[train_set['Birth Year'].isna() & train_set['Accident Year'].notna() & train_set['Age at Injury'].notna()].head()

,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Agreement Reached,Number of Dependents,Days_to_Assembly,Accident Year,Accident Month,Accident Day
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5980545,31.0,N,2022-05-02,N,3.063832,NaN,2022-04-30,NaT,HEALTH & HOSPITAL CORP.,3A. SELF PUBLIC,BRONX,N,NYC,2023-01-09,F,0.0,62.0,IV,99.0,1.0,-9.0,10451,0.0,2.0,27.0,2022.0,4.0,5.0
5488794,31.0,N,2020-05-29,N,0.000000,NaN,2020-06-08,2020-05-20,STARR SPECIALTY INSURANCE,1A. PRIVATE,SUFFOLK,N,HAUPPAUGE,NaN,M,0.0,54.0,UK,45.0,52.0,34.0,34482,0.0,6.0,199.0,2019.0,11.0,12.0
5469588,33.0,N,2020-04-21,Y,3.213783,NaN,2020-05-21,2020-04-08,"POLICE, FIRE, SANITATION",3A. SELF PUBLIC,RICHMOND,N,NYC,2021-09-27,M,1.0,92.0,IV,27.0,10.0,31.0,10308,0.0,6.0,29.0,2020.0,3.0,23.0
5614437,25.0,N,2020-12-10,Y,3.132935,NaN,2020-12-18,2020-11-03,FEDERAL INSURANCE COMPANY,1A. PRIVATE,ERIE,N,BUFFALO,2021-03-09,M,1.0,71.0,UK,98.0,80.0,90.0,02370,0.0,6.0,622.0,2019.0,3.0,29.0
5458247,59.0,N,2020-03-24,N,0.000000,NaN,2020-03-24,NaT,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,WESTCHESTER,N,NYC,NaN,M,0.0,23.0,III,53.0,59.0,33.0,10587,0.0,3.0,613.0,2018.0,7.0,20.0


There are some rows, that eventhough there are no values in the feature `Birth Year`, we can find based on the values in `Age at Injury` and `Accident Year`.

In [42]:
train_set.loc[train_set['Birth Year'].isna() & train_set['Accident Year'].notna() & train_set['Age at Injury'].notna(), 'Birth Year'] = (
    train_set['Accident Year'] - train_set['Age at Injury'])

In [43]:
val_set.loc[val_set['Birth Year'].isna() & val_set['Accident Year'].notna() & val_set['Age at Injury'].notna(), 'Birth Year'] = (
    val_set['Accident Year'] - val_set['Age at Injury'])

Some rows have the same year in `Accident Year` and `Birth Year` and that is inconsistent. Based on that, the values in `Birth Year` will be replaced as missing.

In [44]:
train_set[train_set["Birth Year"] == train_set["Accident Year"]].head(5)

,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Agreement Reached,Number of Dependents,Days_to_Assembly,Accident Year,Accident Month,Accident Day
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5721908,0.0,N,2021-05-14,N,0.0,2021.0,NaT,NaT,AMERICAN GUAR & LIAB INS CO,1A. PRIVATE,CHENANGO,N,BINGHAMTON,NaN,F,0.0,0.0,I,0.0,0.0,0.0,13815,0.0,5.0,74.0,2021.0,3.0,1.0
5578131,0.0,N,2020-10-15,N,0.0,2020.0,2020-08-14,NaT,"GENERAL MOTORS, LLC",4A. SELF PRIVATE,NIAGARA,N,BUFFALO,NaN,F,0.0,0.0,I,99.0,59.0,38.0,NaN,0.0,5.0,248.0,2020.0,2.0,10.0
5482425,0.0,N,2020-05-15,N,NaN,2020.0,NaT,NaT,STATE INSURANCE FUND,2A. SIF,DUTCHESS,N,ALBANY,NaN,F,0.0,61.0,II,0.0,0.0,0.0,12601,0.0,1.0,75.0,2020.0,3.0,1.0
5708974,0.0,N,2021-04-27,N,0.0,2021.0,2021-04-23,NaT,GREAT NECK UNION FREE,3A. SELF PUBLIC,NASSAU,N,NYC,NaN,F,0.0,0.0,IV,74.0,10.0,17.0,11021,0.0,5.0,15.0,2021.0,4.0,12.0
5564192,0.0,N,2020-09-23,Y,0.0,2020.0,NaT,2020-08-20,STATE INSURANCE FUND,2A. SIF,NEW YORK,N,NYC,NaN,F,0.0,62.0,IV,0.0,0.0,0.0,11356,0.0,2.0,246.0,2020.0,1.0,21.0


In [45]:
train_set.loc[train_set["Birth Year"] == train_set["Accident Year"], "Birth Year"] = np.nan

In [46]:
val_set.loc[val_set["Birth Year"] == val_set["Accident Year"], "Birth Year"] = np.nan

Since, some rows have the age at injury different from the difference between the birth year and the year of the accident, those will be updated.

In [47]:
train_set['Age at Injury'] = train_set['Accident Year'] - train_set['Birth Year']

In [48]:
val_set['Age at Injury'] = val_set['Accident Year'] - val_set['Birth Year']

In [49]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
import numpy as np

output_notebook()

cleaned_age = train_set['Age at Injury'].dropna()

hist, edges = np.histogram(cleaned_age, bins=40)

p = figure(title='Histogram of Age at Injury')

p.quad(
    top=hist,
    bottom=0,
    left=edges[:-1],
    right=edges[1:],
    fill_color='#FF8C00',
    line_color='black',      
    line_width=1 
)

# Display the plot in the notebook
show(p)

Loading BokehJS ...

<br>

2) Cyclical Encoding

`Accident Month` and `Accident Day` are inherently cyclical, since those features derived from datetime variables. Therefore, we will proceed to encode these features using a cyclical encoding. 

In [50]:
train_set['Accident Month Sin'] = np.sin(2 * np.pi * train_set['Accident Month'] / 12)
train_set['Accident Month Cos'] = np.cos(2 * np.pi * train_set['Accident Month'] / 12)

val_set['Accident Month Sin'] = np.sin(2 * np.pi * val_set['Accident Month'] / 12)
val_set['Accident Month Cos'] = np.cos(2 * np.pi * val_set['Accident Month'] / 12)

test['Accident Month Sin'] = np.sin(2 * np.pi * test['Accident Month'] / 12)
test['Accident Month Cos'] = np.cos(2 * np.pi * test['Accident Month'] / 12)

In [51]:
train_set['Accident Day Sin'] = np.sin(2 * np.pi * train_set['Accident Day'] / 31)
train_set['Accident Day Cos'] = np.cos(2 * np.pi * train_set['Accident Day'] / 31)

val_set['Accident Day Sin'] = np.sin(2 * np.pi * val_set['Accident Day'] / 31)
val_set['Accident Day Cos'] = np.cos(2 * np.pi * val_set['Accident Day'] / 31)

test['Accident Day Sin'] = np.sin(2 * np.pi * test['Accident Day'] / 31)
test['Accident Day Cos'] = np.cos(2 * np.pi * test['Accident Day'] / 31)

Removing the columns `Accident Day` and `Accident Month`

In [ ]:
train_set = train_set.drop(columns=['Accident Month', 'Accident Day'])

<br>

#### 4.1.2. `Zip Code`

As previously mentioned, there were some Zip codes that were inconsistent, therefore we will address them.

In [20]:
len(train_set["Zip Code"].unique())

9135

In [21]:
len(train_set[train_set["Zip Code"] == '10567'])

406

In [22]:
len(train_set[train_set["Zip Code"] == 10567.0])

0

The `fix_zip_code` function standardizes zip codes by converting any float values to integers and then back to strings. This ensures consistent formatting across all zip code entries.

In [23]:
def fix_zip_code(x):
    if isinstance(x, float) and x.is_integer():
        return str(int(x))[:5]
    if isinstance(x, str) and x.isnumeric():
        return str(int(x))[:5]
    try:
        return(str(int(x))[:5])
    except:
        warnings.warn("x cannot be turned into a string of 5 characters", UserWarning)

In [24]:
train_set['Zip Code'] = train_set['Zip Code'].apply(fix_zip_code)
val_set['Zip Code'] = val_set['Zip Code'].apply(fix_zip_code)

/var/folders/pw/sbpycr1d17b1j4255gwjl17c0000gn/T/ipykernel_41452/2114121942.py:9: UserWarning: x cannot be turned into a string of 5 characters
  warnings.warn("x cannot be turned into a string of 5 characters", UserWarning)


Checking if it worked.

In [25]:
len(train_set["Zip Code"].unique())

7304

In [26]:
len(train_set[train_set["Zip Code"] == '10567'])

406

In [27]:
len(train_set[train_set["Zip Code"] == 10567.0])

0

Now, the values for the `Zip Code` feature are standardized.

In [53]:
train_set.select_dtypes(include='number')

,Age at Injury,Average Weekly Wage,Birth Year,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Agreement Reached,Number of Dependents,Days_to_Assembly,Accident Year,Accident Month,Accident Day,Accident Month Sin,Accident Month Cos,Accident Day Sin,Accident Day Cos
Claim Identifier,,,,,,,,,,,,,,,,,,
5785935,50.0,2.872191,1971.0,2.0,31.0,29.0,59.0,42.0,0.0,6.0,5.0,2021.0,8.0,5.0,-8.660254e-01,-5.000000e-01,0.848644,0.528964
5980545,31.0,3.063832,1991.0,0.0,62.0,99.0,1.0,-9.0,0.0,2.0,27.0,2022.0,4.0,5.0,8.660254e-01,-5.000000e-01,0.848644,0.528964
5552635,44.0,0.000000,1976.0,0.0,49.0,80.0,52.0,56.0,0.0,3.0,3.0,2020.0,9.0,1.0,-1.000000e+00,-1.836970e-16,0.201299,0.979530
5758039,25.0,0.000000,1996.0,0.0,44.0,68.0,10.0,11.0,0.0,1.0,22.0,2021.0,6.0,10.0,1.224647e-16,-1.000000e+00,0.897805,-0.440394
5951382,42.0,2.399674,1980.0,0.0,48.0,45.0,49.0,-9.0,0.0,4.0,58.0,2022.0,1.0,26.0,5.000000e-01,8.660254e-01,-0.848644,0.528964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5837651,26.0,3.095365,1995.0,1.0,62.0,98.0,80.0,41.0,0.0,6.0,4.0,2021.0,10.0,15.0,-8.660254e-01,5.000000e-01,0.101168,-0.994869
5781926,55.0,2.986888,1966.0,7.0,62.0,74.0,10.0,90.0,0.0,1.0,7.0,2021.0,7.0,28.0,-5.000000e-01,-8.660254e-01,-0.571268,0.820763
5890060,57.0,2.900416,1964.0,0.0,81.0,53.0,52.0,53.0,0.0,5.0,54.0,2021.0,11.0,11.0,-5.000000e-01,8.660254e-01,0.790776,-0.612106


<br>

##### 4.2 Feature engineering

*Dummy Features*

To create binary indicators for whether each form was received, we'll define new columns: `Received_Employer_report` and `Received_employee_claim`. Each will be set to 1 if the corresponding date is present, and 0 if it's missing (already replaced with 0).

In [28]:
train_set["Received_C2"] = train_set["C-2 Date"].apply(lambda x: 0 if pd.isna(x) else 1)
val_set["Received_C2"] = val_set["C-2 Date"].apply(lambda x: 0 if pd.isna(x) else 1)
test["Received_C2"] = test["C-2 Date"].apply(lambda x: 0 if pd.isna(x) else 1)

In [29]:
train_set["Received_C3"] = train_set["C-3 Date"].apply(lambda x: 0 if pd.isna(x) else 1)
val_set["Received_C3"] = val_set["C-3 Date"].apply(lambda x: 0 if pd.isna(x) else 1)
test["Received_C3"] = test["C-3 Date"].apply(lambda x: 0 if pd.isna(x) else 1)

The same will be applied to indicate if a hearing was held. We'll create a new column, `Hearing_held`, set to 1 if the `First Hearing Date` happened and 0 if did not (already replaced with 0).

In [30]:
train_set["Hearing_held"] = train_set["First Hearing Date"].apply(lambda x: 0 if pd.isna(x) else 1)
val_set["Hearing_held"] = val_set["First Hearing Date"].apply(lambda x: 0 if pd.isna(x) else 1)
test["Hearing_held"] = test["First Hearing Date"].apply(lambda x: 0 if pd.isna(x) else 1)

<br>

*Categorical encoding*

In this step, we are going to encode some columns using the One Hot Encoder.

Some features that are binary categorical variables, like yes (Y) and no (N), we will convert it to 0 and 1. One of them will be discarded.

In [31]:
train_set = pd.get_dummies(train_set, columns=['Attorney/Representative', 'COVID-19 Indicator'], drop_first=True, dtype=int)
val_set = pd.get_dummies(val_set, columns=['Attorney/Representative', 'COVID-19 Indicator'], drop_first=True, dtype=int)
test = pd.get_dummies(test, columns=['Attorney/Representative', 'COVID-19 Indicator'], drop_first=True, dtype=int)

<br>Defining the columns to encode.

In [32]:
columns_to_encode = ['Alternative Dispute Resolution', 'Gender']

Initializing the encoder and fitting it on the training set. This encoder will learns the unique categories present in the training set.

In [33]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit(train_set[columns_to_encode])

Transforming the `train_set`, `val_set`, and `test_set` with the fitted encoder. This ensures that the columns are encoded consistently across all sets.

In [34]:
train_encoded = encoder.transform(train_set[columns_to_encode])
validation_encoded = encoder.transform(val_set[columns_to_encode])
test_encoded = encoder.transform(test[columns_to_encode])

Converting the arrays into DataFrames with the appropriate column names.

In [35]:
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(columns_to_encode), index=train_set.index)
validation_encoded_df = pd.DataFrame(validation_encoded, columns=encoder.get_feature_names_out(columns_to_encode), index=val_set.index)
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(columns_to_encode), index=test.index)

Dropping the original columns.

In [36]:
train_set.drop(columns=columns_to_encode, inplace=True)
train_set = pd.concat([train_set, train_encoded_df], axis=1)

In [37]:
val_set.drop(columns=columns_to_encode, inplace=True)
val_set = pd.concat([val_set, validation_encoded_df], axis=1)

In [38]:
test.drop(columns=columns_to_encode, inplace=True)
test = pd.concat([test, test_encoded_df], axis=1)

Dropping the features `Alternative Dispute Resolution_U` and `Gender_X`, in order to avoid multicollinearity.

In [39]:
train_set.drop(columns='Alternative Dispute Resolution_U', inplace=True)
val_set.drop(columns='Alternative Dispute Resolution_U', inplace=True)
test.drop(columns='Alternative Dispute Resolution_U', inplace=True)

In [40]:
train_set.drop(columns='Gender_X', inplace=True)
val_set.drop(columns='Gender_X', inplace=True)
test.drop(columns='Gender_X', inplace=True)

<br>

Frequency encoding

`Industry Code`

In [41]:
frequency_map_ic = train_set['Industry Code'].value_counts(normalize=True)

In [42]:
train_set['Industry Code'] = train_set['Industry Code'].map(frequency_map_ic)
val_set['Industry Code'] = val_set['Industry Code'].map(frequency_map_ic)
test['Industry Code'] = test['Industry Code'].map(frequency_map_ic)

<br>

`WCIO Cause of Injury Code`

In [43]:
frequency_map_wcio_ic = train_set['WCIO Cause of Injury Code'].value_counts(normalize=True)

In [44]:
train_set['WCIO Cause of Injury Code'] = train_set['WCIO Cause of Injury Code'].map(frequency_map_wcio_ic)
val_set['WCIO Cause of Injury Code'] = val_set['WCIO Cause of Injury Code'].map(frequency_map_wcio_ic)
test['WCIO Cause of Injury Code'] = test['WCIO Cause of Injury Code'].map(frequency_map_wcio_ic)

<br>

`WCIO Nature of Injury Code`

In [45]:
frequency_map_wcio_nic = train_set['WCIO Nature of Injury Code'].value_counts(normalize=True)

In [46]:
train_set['WCIO Nature of Injury Code'] = train_set['WCIO Nature of Injury Code'].map(frequency_map_wcio_nic)
val_set['WCIO Nature of Injury Code'] = val_set['WCIO Nature of Injury Code'].map(frequency_map_wcio_nic)
test['WCIO Nature of Injury Code'] = test['WCIO Nature of Injury Code'].map(frequency_map_wcio_nic)

<br>

`WCIO Part Of Body Code`

In [47]:
frequency_map_wcio_pbc = train_set['WCIO Part Of Body Code'].value_counts(normalize=True)

In [48]:
train_set['WCIO Part Of Body Code'] = train_set['WCIO Part Of Body Code'].map(frequency_map_wcio_pbc)
val_set['WCIO Part Of Body Code'] = val_set['WCIO Part Of Body Code'].map(frequency_map_wcio_pbc)
test['WCIO Part Of Body Code'] = test['WCIO Part Of Body Code'].map(frequency_map_wcio_pbc)

<br>

`Medical Fee Region`

In [49]:
frequency_map_mfr = train_set['Medical Fee Region'].value_counts(normalize=True)

In [50]:
train_set['Medical Fee Region'] = train_set['Medical Fee Region'].map(frequency_map_mfr)
val_set['Medical Fee Region'] = val_set['Medical Fee Region'].map(frequency_map_mfr)
test['Medical Fee Region'] = test['Medical Fee Region'].map(frequency_map_mfr)

<br>

`Carrier Type`

In [51]:
frequency_map_ct = train_set['Carrier Type'].value_counts(normalize=True)

In [52]:
train_set['Carrier Type'] = train_set['Carrier Type'].map(frequency_map_ct)
val_set['Carrier Type'] = val_set['Carrier Type'].map(frequency_map_ct)
test['Carrier Type'] = test['Carrier Type'].map(frequency_map_ct)

<br>

*Encoding the target feature*

Given that the target variable is ordinal and categorical, we are going to encode it.

In [53]:
injury_type_mapping = {
    '1. CANCELLED': 1,
    '2. NON-COMP': 2,
    '3. MED ONLY': 3,
    '4. TEMPORARY': 4,
    '5. PPD SCH LOSS': 5,
    '6. PPD NSL': 6,
    '7. PTD': 7,
    '8. DEATH': 8
}

In [54]:
y_train_encoded = y_train.map(injury_type_mapping)
y_val_encoded = y_test.map(injury_type_mapping)

<br>

#### 4.3 Checking Outliers

In [55]:
"""# Set up the matplotlib figure
plt.figure(figsize=(15, 8))  

# Create a grid of subplots for numeric columns
for i, column in enumerate(numeric_columns, 1):
    plt.subplot(4, 4, i)  
    sns.boxplot(data=train_set[column], color='steelblue')
    plt.title(column)  

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()"""

"# Set up the matplotlib figure\nplt.figure(figsize=(15, 8))  \n\n# Create a grid of subplots for numeric columns\nfor i, column in enumerate(numeric_columns, 1):\n    plt.subplot(4, 4, i)  \n    sns.boxplot(data=train_set[column], color='steelblue')\n    plt.title(column)  \n\n# Adjust layout to prevent overlap\nplt.tight_layout()\n\n# Show the plot\nplt.show()"

Given the presence of outliers in some features, we will apply capping to mitigate their influence. This will be achieved by scaling the `train_set` using the Robust Scaler, which is less sensitive to outliers and ensures that the scaling is based on the interquartile range (IQR) rather than the mean and standard deviation.

<br>

#### 4.1.1 Filling the missing values

As noticed the data has some missing data, therefore we will fill the missing values with the median and mode for numerical and categorical features, respectively.

In [ ]:
train_set.isna().sum()

Accident Date                       2959
Age at Injury                          0
Alternative Dispute Resolution         0
Assembly Date                          0
Attorney/Representative                0
Average Weekly Wage                22968
Birth Year                         43323
C-2 Date                           11590
C-3 Date                          309591
Carrier Name                           0
Carrier Type                           0
County of Injury                       0
COVID-19 Indicator                     0
District Name                          0
First Hearing Date                338649
Gender                                 0
IME-4 Count                            0
Industry Code                          0
Medical Fee Region                     0
WCIO Cause of Injury Code              0
WCIO Nature of Injury Code             0
WCIO Part Of Body Code                 0
Zip Code                           22869
Agreement Reached                      0
Number of Depend

In [ ]:
col_fill_median = ['Age at Injury', 'Average Weekly Wage', 'Birth Year', 'Accident Year']

In [ ]:
for col in col_fill_median:
    median_value = train_set[col].median()
    
    train_set[col].fillna(median_value, inplace=True)
    val_set[col].fillna(median_value, inplace=True)
    test[col].fillna(median_value, inplace=True)

KeyError: 'Accident Year'

In [ ]:
col_fill_mode = ['C-2 Date', 'C-3 Date', 'First Hearing Date', 'Zip Code']

In [ ]:
for col in col_fill_mode:
    mode_value = train_set[col].mode()[0]
    
    train_set[col].fillna(mode_value, inplace=True)
    val_set[col].fillna(mode_value, inplace=True)
    test[col].fillna(mode_value, inplace=True)

/var/folders/pw/sbpycr1d17b1j4255gwjl17c0000gn/T/ipykernel_41452/1051533434.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_set[col].fillna(mode_value, inplace=True)
/var/folders/pw/sbpycr1d17b1j4255gwjl17c0000gn/T/ipykernel_41452/1051533434.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

<br>

#### 3.4 Feature Scaling

Before applying the scaler is necessary to first split the `train_set`, `val_set` and `test` into a numerical and categorical set. In this way, we can apply the scaler in the numerical features.

In [61]:
train_numerical = train_set.select_dtypes(include='number')
train_categorical = train_set.select_dtypes(include='object')

In [67]:
val_numerical = val_set.select_dtypes(include='number')
val_categorical = val_set.select_dtypes(include='object')

In [72]:
test_numerical = test.select_dtypes(include='number')
test_categorical = test.select_dtypes(include='object')

Getting the non boolean features so we can scale these features.

In [66]:
train_num_excluding_booleans = train_numerical.loc[:, train_numerical.nunique() > 2]

In [68]:
val_num_excluding_booleans = val_numerical.loc[:, val_numerical.nunique() > 2]

In [73]:
test_num_excluding_booleans = test_numerical.loc[:, test_numerical.nunique() > 2]

<br>

Scaler

In [74]:
scaler = RobustScaler().fit(train_num_excluding_booleans)

Fitting the scaler on the numerical training data and transforming the training data. Also, converts it to a dataframe.

In [75]:
X_train_rb = scaler.transform(train_num_excluding_booleans) 
X_train_rb = pd.DataFrame(X_train_rb, columns = train_num_excluding_booleans.columns).set_index(train_num_excluding_booleans.index)

Transforming the validation data using the same scaler and converting to a dataframe.

In [77]:
X_val_rb = scaler.transform(val_num_excluding_booleans) 
X_val_rb = pd.DataFrame(val_num_excluding_booleans, columns = val_num_excluding_booleans.columns).set_index(val_num_excluding_booleans.index)

Transforming the test data using the same scaler and converting to a dataframe.

In [78]:
test_rb = scaler.transform(test_num_excluding_booleans) 
test_rb = pd.DataFrame(test_num_excluding_booleans, columns = test_num_excluding_booleans.columns).set_index(test_num_excluding_booleans.index)

<br>

Concatenate the datasets

In [81]:
# Extract boolean columns from the original datasets
train_num_booleans = train_numerical.loc[:, train_numerical.nunique() <= 2]
val_num_booleans = val_numerical.loc[:, val_numerical.nunique() <= 2]
test_num_booleans = test_numerical.loc[:, test_numerical.nunique() <= 2]

In [83]:
X_train_full = pd.concat([X_train_rb, train_num_booleans, train_categorical], axis=1)
X_val_full = pd.concat([X_val_rb, val_num_booleans, val_categorical], axis=1)
test_full = pd.concat([test_rb, test_num_booleans, test_categorical], axis=1)

In [84]:
X_train_full

,Age at Injury,Average Weekly Wage,Birth Year,Carrier Type,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Agreement Reached,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_N,Alternative Dispute Resolution_Y,Gender_F,Gender_M,Gender_U,Assembly Date,C-2 Date,C-3 Date,Carrier Name,County of Injury,District Name,First Hearing Date,Zip Code
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5785935,0.347826,0.989528,-0.318182,0.938705,2.0,-0.447372,0.720134,0.582815,-0.149595,1.095184,0.75,0.0,0.0,1,1,1,1,0,1.0,0.0,1.0,0.0,0.0,2021-08-10,2021-08-10,2021-08-30,PROPERTY AND CASUALTY,QUEENS,NYC,2022-04-15,11432
5980545,-0.478261,1.055553,0.590909,0.000000,0.0,0.944149,0.720134,0.268416,-0.380814,0.726994,-0.25,0.5,0.0,1,1,1,0,0,1.0,0.0,1.0,0.0,0.0,2022-05-02,2022-04-30,2020-02-10,HEALTH & HOSPITAL CORP.,BRONX,NYC,2023-01-09,10451
5552635,0.086957,0.000000,-0.090909,0.938705,0.0,-0.358211,0.720134,-0.619590,0.748100,-0.147965,0.00,-0.5,0.0,1,1,1,0,0,1.0,0.0,0.0,1.0,0.0,2020-09-04,2020-09-04,2020-02-10,AMERICAN ZURICH INSURANCE CO,KINGS,NYC,2022-04-15,11203
5758039,-0.739130,0.000000,0.818182,-0.395841,0.0,0.000000,0.720134,-0.270315,0.417989,-0.559172,-0.50,0.0,0.0,1,1,1,0,0,1.0,0.0,0.0,1.0,0.0,2021-07-02,2021-07-02,2020-02-10,"NORDSTROM, INC.",KINGS,NYC,2022-04-15,11237
5951382,0.000000,0.826736,0.090909,-0.395841,0.0,-0.083434,0.720134,0.000000,0.000000,0.726994,0.25,0.5,0.0,1,1,1,1,0,1.0,0.0,0.0,1.0,0.0,2022-03-25,2022-03-30,2022-03-15,NEW YORK BLACK CAR OPERATORS',NASSAU,NYC,2022-12-28,11003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5837651,-0.695652,1.066416,0.772727,0.938705,1.0,0.944149,-0.452118,-0.566298,-0.404784,-0.549172,0.75,0.0,0.0,1,1,1,1,0,1.0,0.0,0.0,1.0,0.0,2021-10-19,2021-10-19,2021-11-10,INDEMNITY INSURANCE CO OF,WESTCHESTER,NYC,2022-01-24,10805
5781926,0.565217,1.029044,-0.545455,0.938705,7.0,0.944149,0.720134,0.938996,0.417989,0.000000,-0.50,0.0,0.0,1,1,1,1,0,1.0,0.0,0.0,1.0,0.0,2021-08-04,2021-08-04,2021-08-19,REDWOOD FIRE & CASUALTY,QUEENS,NYC,2022-03-07,11429
5890060,0.652174,0.999252,-0.636364,0.938705,0.0,-0.405647,0.000000,-0.220534,0.748100,0.957785,0.50,0.0,0.0,1,1,1,0,0,1.0,0.0,1.0,0.0,0.0,2022-01-04,2022-01-04,2020-02-10,UTICA MUTUAL INS CO,ERIE,BUFFALO,2022-04-15,14127
